In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('kaz_df.csv')

In [ ]:
import pandas as pd

# 1. Убираем пустые / None / NaN / слишком короткие строки
df_clean = df[df["text_original"].astype(str).str.strip().str.len() > 0].copy()

# 2. Убираем дубликаты
df_clean = df_clean.drop_duplicates(subset=["text_original"]).reset_index(drop=True)

# 3. Сэмплируем 3000 случайных строк
golden_df = df_clean.sample(3000, random_state=42).reset_index(drop=True)

# 4. Сохраняем
golden_df.to_csv("golden_sample_3000.csv", index=False)
golden_df.to_json("golden_sample_3000.jsonl", orient="records", lines=True)

golden_df.head()

In [ ]:
import os
from dotenv import load_dotenv

# ВАЖНО: перезаписываем системные переменные значениями из .env
load_dotenv(override=True)

api_key = os.getenv("OPENAI_API_KEY")

print("repr(api_key):", repr(api_key))
print("len(api_key):", len(api_key) if api_key is not None else None)

if api_key is None:
    print("❌ OPENAI_API_KEY не найден")
elif not api_key.startswith("sk-"):
    print("❌ Ключ не начинается с 'sk-'")
else:
    print("✅ Ключ найден, первые 12 символов:", api_key[:12])


In [ ]:
import os
import json
import time
import pandas as pd
from typing import List, Dict, Any

from openai import OpenAI

from dotenv import load_dotenv

# Загружаем .env
load_dotenv()

# Читаем ключ
api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=api_key)


# === ПАРАМЕТРЫ ===
INPUT_CSV = "golden_sample_3000.csv"       # входной голденсет
OUTPUT_JSONL = "golden_annotated.jsonl"    # аннотации построчно
OUTPUT_CSV = "golden_annotated.csv"        # финальная табличка
TEXT_COL = "text_original"                 # имя колонки с текстом
MODEL_NAME = "gpt-4.1-mini"                # или gpt-4.1, gpt-4o-mini и т.п.
BATCH_SIZE = 30                            # сколько комментариев отправляем за раз
SLEEP_BETWEEN_CALLS = 0.5                  # пауза между запросами, чтобы не заддосить API


# === СИСТЕМНЫЙ ПРОМПТ ===
SYSTEM_PROMPT = """
Сен қазақ тіліндегі пікірлерді модерациялайтын көмекшісің.
Саған әр пікір (комментарий) мәтіні беріледі. Сенің міндетің – оны токсиктілік
бойынша келесі бинарлық белгілермен белгілеу (true/false).

Анықтамалар:
- is_toxic: Пікірде кез келген токсиктілік бар ма? Егер төмендегі кез келген
  категория true болса, is_toxic = true.
- toxic: Жалпы агрессивті, құрметсіз, өшпенді реңк (міндетті түрде балағат сөз
  болмауы мүмкін).
- obscene: Балағат/боқауыз сөздер, өте дөрекі сленг, әдепсіз сөздер.
- threat: Белгілі бір адамға немесе адамдар тобына физикалық зиян келтіремін
  деп қорқыту (мысалы, ұрып жіберемін, өлтіремін деген сияқты).
- insult: Белгілі бір адамды немесе адамдарды қорлау, кемсіту (ақымақ, мал, т.б.).
- hate: Әлеуметтік топқа (ұлт, нәсіл, дін, жыныс, ориентация, т.б.) бағытталған
  өшпенділік, кемсіту, адам ретінде құқығын жоққа шығару.

Әр пікір үшін:
- Әр белгіге true/false бер,
- Қысқа explanation (2–3 сөйлемнен аспасын) жазып бер: неге дәл осылай
  белгілегеніңді түсіндір. Егер пікір бейтарап (токсикті емес) болса,
  оны да жазып жібер (мысалы, "бейтарап пікір, агрессия жоқ" деген сияқты).

МАҢЫЗДЫ:
- Тек JSON қайтар. Басқа ешқандай мәтін жазба.
"""


def build_user_prompt(items: List[Dict[str, Any]]) -> str:
    """
    items: список словарей вида {"id": ..., "text": ...}
    Формируем пользовательский промпт с несколькими комментариями.
    """
    lines = [
        "Төменде бірнеше пікір берілген. Әр пікір үшін төмендегі JSON форматында жауап бер:",
        "",
        "{",
        '  "items": [',
        '    {',
        '      "id": "<id>",',
        '      "is_toxic": true/false,',
        '      "toxic": true/false,',
        '      "obscene": true/false,',
        '      "threat": true/false,',
        '      "insult": true/false,',
        '      "hate": true/false,',
        '      "explanation": "<қысқа түсіндірме>"',
        '    },',
        '    ...',
        '  ]',
        "}",
        "",
        "Пікірлер тізімі:"
    ]
    for it in items:
        lines.append(f'ID: {it["id"]}')
        # Экранируем переносы строк на всякий случай
        text_clean = str(it["text"]).replace("\n", " ").strip()
        lines.append(f'Text: {text_clean}')
        lines.append("---")

    return "\n".join(lines)


def annotate_batch(items: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Делает один запрос к OpenAI для батча комментариев
    и возвращает список аннотаций по каждому id.
    """
    user_prompt = build_user_prompt(items)

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.0,
    )

    content = response.choices[0].message.content

    # Парсим JSON
    try:
        data = json.loads(content)
    except json.JSONDecodeError:
        # Если модель вдруг вернула мусор — можно логировать и/или ретраить
        raise RuntimeError(f"JSON parsing error. Raw content:\n{content}")

    if "items" not in data or not isinstance(data["items"], list):
        raise RuntimeError(f"Unexpected JSON structure: {data}")

    return data["items"]


def main():
    df = pd.read_csv(INPUT_CSV)

    # Готовим список элементов для аннотации
    records = []
    for idx, row in df.iterrows():
        text = row[TEXT_COL]
        if not isinstance(text, str) or not text.strip():
            continue
        records.append({
            "id": int(idx),   # можно заменить на row["id"], если у тебя есть явный id
            "text": text.strip(),
        })

    print(f"Всего комментариев для разметки: {len(records)}")

    # Проходимся по батчам и пишем результат в JSONL
    annotated = []
    with open(OUTPUT_JSONL, "w", encoding="utf-8") as f_out:
        for i in range(0, len(records), BATCH_SIZE):
            batch = records[i:i + BATCH_SIZE]
            print(f"Обрабатываем батч {i}–{i+len(batch)-1}...")

            try:
                batch_labels = annotate_batch(batch)
            except Exception as e:
                print(f"Ошибка при аннотации батча {i}: {e}")
                # Можно сделать ретрай, паузу и т.д.
                continue

            # Сохраняем в JSONL по одной строке
            for item in batch_labels:
                f_out.write(json.dumps(item, ensure_ascii=False) + "\n")
                annotated.append(item)

            time.sleep(SLEEP_BETWEEN_CALLS)

    print(f"Всего размечено: {len(annotated)}")

    # Собираем всё в DataFrame и мёрджим с исходным golden_df
    ann_df = pd.DataFrame(annotated)

    # Приводим логические поля к bool
    bool_cols = ["is_toxic", "toxic", "obscene", "threat", "insult", "hate"]
    for col in bool_cols:
        if col in ann_df.columns:
            ann_df[col] = ann_df[col].astype(bool)

    # Соединяем по id
    df_golden = df.reset_index().rename(columns={"index": "id"})
    merged = df_golden.merge(ann_df, on="id", how="inner")

    merged.to_csv(OUTPUT_CSV, index=False)
    print(f"Аннотированный голденсет сохранён в {OUTPUT_CSV}")



In [ ]:
main()

In [ ]:
import pandas as pd
import json

# 1. Оригинальный golden set
df_golden = pd.read_csv("golden_sample_3000.csv")
df_golden = df_golden.reset_index().rename(columns={"index": "id"})

# 2. Загрузка аннотаций (если ты их сохранял в JSONL по одной строке)
ann_df = pd.read_json("golden_annotated.jsonl", lines=True)

# 3. Приводим типы id к строке с обеих сторон
df_golden["id"] = df_golden["id"].astype(str)
ann_df["id"] = ann_df["id"].astype(str)

# 4. Мёрджим
merged = df_golden.merge(ann_df, on="id", how="inner")

print(len(df_golden), len(ann_df), len(merged))
merged.to_csv("golden_annotated_merged.csv", index=False)

In [ ]:
import pandas as pd

merged = pd.read_csv("golden_annotated_merged.csv")
df = pd.read_csv('kaz_df.csv')
df = df[~df.comment_id.isin(merged.comment_id)]

In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

MODEL_KZ_CHECKPOINT = "./kaz_roberta_toxic_kz"  # <-- твой путь

tokenizer_kz = AutoTokenizer.from_pretrained(MODEL_KZ_CHECKPOINT)
model_kz = AutoModelForSequenceClassification.from_pretrained(MODEL_KZ_CHECKPOINT)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_kz.to(device)
model_kz.eval()

# ПОРЯДОК ЛЕЙБЛОВ ДОЛЖЕН СОВПАДАТЬ С ТЕМ, КАК ТЫ УЧИЛ МОДЕЛЬ
tox_aspect_names_kz = ["toxic", "obscene", "threat", "insult", "hate"]


In [ ]:
def text2toxicity_batch_kz(texts, aggregate=False):
    """
    texts: список сырых значений (строки, NaN, числа и т.д.)
    aggregate=False -> возвращает матрицу [batch, 5] по аспектам
    aggregate=True  -> возвращает вектор агрегированного score [batch]
                      (вероятность, что текст токсичен по хоть одному аспекту)
    """
    # чистим вход: всё к строке, NaN/None -> ""
    cleaned_texts = []
    for t in texts:
        if t is None or (isinstance(t, float) and pd.isna(t)):
            cleaned_texts.append("")
        else:
            cleaned_texts.append(str(t))

    with torch.no_grad():
        inputs = tokenizer_kz(
            cleaned_texts,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512,
        ).to(device)

        logits = model_kz(**inputs).logits
        proba = torch.sigmoid(logits).cpu().numpy()  # shape: [batch, 5]

    if aggregate:
        # union probability: 1 - prod(1 - p_i) по всем 5 аспектам
        agg = 1.0 - np.prod(1.0 - proba, axis=1)
        return agg  # shape: [batch]

    return proba  # shape: [batch, 5]


In [ ]:
def text2toxicity_batch_kz_compat(texts, aggregate=False):
    """
    Совместимый по форме вариант:
    aggregate=True -> 1 - p[:,0] * (1 - p[:,-1])
    НО: у тебя p[:,0] = P(toxic), p[:,-1] = P(hate), это НЕ то же самое, что в cointegrated.
    """
    cleaned_texts = []
    for t in texts:
        if t is None or (isinstance(t, float) and pd.isna(t)):
            cleaned_texts.append("")
        else:
            cleaned_texts.append(str(t))

    with torch.no_grad():
        inputs = tokenizer_kz(
            cleaned_texts,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512,
        ).to(device)

        logits = model_kz(**inputs).logits
        proba = torch.sigmoid(logits).cpu().numpy()

    if aggregate:
        agg = 1.0 - proba[:, 0] * (1.0 - proba[:, -1])
        return agg
    return proba


In [ ]:
def add_toxicity_to_df_kz(
    df,
    text_col="text_original",
    batch_size=64,
    add_aspects=True,
    use_compat_formula=False,
):
    """
    df: DataFrame с текстами
    text_col: колонка с текстом
    batch_size: размер батча
    add_aspects: если True — создаст колонки по аспектам
    use_compat_formula: если True — использовать старую формулу cointegrated
                        (1 - p[:,0] * (1 - p[:,-1]));
                        если False — union probability по всем аспектам.
    """
    all_agg_scores = []
    all_aspects = []

    fn = text2toxicity_batch_kz_compat if use_compat_formula else text2toxicity_batch_kz

    for i in tqdm(range(0, len(df), batch_size)):
        batch_texts = df[text_col].iloc[i:i + batch_size].tolist()

        # всегда берём proba аспектов:
        batch_proba = text2toxicity_batch_kz(batch_texts, aggregate=False)  # [B, 5]

        # агрегат в зависимости от режима
        if use_compat_formula:
            batch_agg = 1.0 - batch_proba[:, 0] * (1.0 - batch_proba[:, -1])
        else:
            batch_agg = 1.0 - np.prod(1.0 - batch_proba, axis=1)

        all_agg_scores.extend(batch_agg.tolist())
        all_aspects.append(batch_proba)

    # агрегированный score
    df["toxicity_score_kz"] = all_agg_scores

    if add_aspects:
        aspects_arr = np.vstack(all_aspects)  # [N, 5]
        for j, name in enumerate(tox_aspect_names_kz):
            df[f"tox_kz_{name}"] = aspects_arr[:, j]

    return df


In [ ]:
# df уже есть, тексты в "text_original"
df = add_toxicity_to_df_kz(
    df,
    text_col="text_original",
    batch_size=64,
    add_aspects=True,
    use_compat_formula=False,  # рекомендую False
)


In [ ]:
df.sort_values('toxicity_score_kz')